# Preprocessing with FeatureUnion example

The easyflow.preprocessing module contains functionality similar to what sklearn does with its Pipeline, FeatureUnion and ColumnTransformer does. 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Normalization, IntegerLookup, StringLookup
from keras.utils.vis_utils import plot_model

In [2]:
# local imports
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.preprocessing.pipeline import FeaturePreprocessorUnion
from easyflow.preprocessing import FeatureInputLayer, SequentialPreprocessingChainer

## Read in data and map as tf.data.Dataset

Use the TensorflowDataMapper class to map pandas data frame to a tf.data.Dataset type

In [3]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)
labels = dataframe.pop("target")

batch_size = 32
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(dataframe, labels)
train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

In [4]:
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal


## Set constants

In [5]:
NUMERICAL_FEATURES = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']
CATEGORICAL_FEATURES = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca']
# thal is represented as a string
STRING_CATEGORICAL_FEATURES = ['thal']

dtype_mapper = {
    "age": tf.float32,
    "sex": tf.float32,
    "cp": tf.float32,
    "trestbps": tf.float32,
    "chol": tf.float32,
    "fbs": tf.float32,
    "restecg": tf.float32,
    "thalach": tf.float32,
    "exang": tf.float32,
    "oldpeak": tf.float32,
    "slope": tf.float32,
    "ca": tf.float32,
    "thal": tf.string,
}

## Setup Preprocessing layer using FeatureUnion

Use Encoder and SequentialEncoder to preprocess features by putting everything in a FeatureUnion object

In [6]:
feature_preprocessor_list = [
    ('numeric_encoder', Normalization(), NUMERICAL_FEATURES),
    ('categorical_encoder', IntegerLookup(output_mode='binary'), CATEGORICAL_FEATURES),
    # For feature thal we first need to run StringLookup followed by a IntegerLookup layer
    ('string_encoder', SequentialPreprocessingChainer([StringLookup(), IntegerLookup(output_mode='binary')]), STRING_CATEGORICAL_FEATURES)
]

In [7]:
preprocessor = FeaturePreprocessorUnion(feature_preprocessor_list)
preprocessor.adapt(train_data_set)

feature_layer_inputs = FeatureInputLayer(dtype_mapper)
preprocessing_layer = preprocessor(feature_layer_inputs)

2022-04-03 09:46:08.041280: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## Compile and Train model

In [8]:
# setup simple network
x = tf.keras.layers.Dense(128, activation="relu")(preprocessing_layer)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=feature_layer_inputs, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

In [9]:
history=model.fit(train_data_set, validation_data=val_data_set, epochs=10)

Epoch 1/10
8/8 [==============================] - 0s 24ms/step - loss: 0.7420 - accuracy: 0.4802 - auc: 0.4561 - val_loss: 0.6541 - val_accuracy: 0.6447 - val_auc: 0.5888
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 0.5928 - accuracy: 0.6696 - auc: 0.6421 - val_loss: 0.5414 - val_accuracy: 0.7500 - val_auc: 0.8165
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 0.5112 - accuracy: 0.7841 - auc: 0.7898 - val_loss: 0.4577 - val_accuracy: 0.8421 - val_auc: 0.8809
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4621 - accuracy: 0.7974 - auc: 0.8496 - val_loss: 0.4121 - val_accuracy: 0.8947 - val_auc: 0.8584
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4319 - accuracy: 0.8062 - auc: 0.8686 - val_loss: 0.4394 - val_accuracy: 0.7500 - val_auc: 0.9122
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 0.4640 - accuracy: 0.7753 - auc: 0.7985 - val_loss: 0.3918 - val_accuracy: 0.85

## Save and Load model

Save and load model setup with FeatureUnion

In [12]:
model.save('model.h5', save_format="h5")
del model

NotImplementedError: 
Layer FeaturePreprocessorUnion has arguments ['self', 'feature_preprocessor_list']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config

In [ ]:
loaded_model = tf.keras.models.load_model("model")

In [ ]:
dict(zip(loaded_model.metrics_names, loaded_model.evaluate(val_data_set)))
